## Development-Kit Tutorial for Zenseact Open Dataset
This notebook aims to introduce the ZodFrames & ZodSequences classes, which are helper classes to interact with the Frames and Sequences subsets of the Zenseact Open Dataset (ZOD) respecively. It will highlight some basic functionality that later can be used to build dataloaders in for example PyTorch.

This notebook also aims to give a brief introduction to the which annotations exist and how to visualization them. 

#### The dataset includes data from 3 sensor modalities and calibrations for each sensor:  
1. **Camera** - Anonymized (license plates and faces) front camera images. Available anonymization methods are:
    - blur (Blur)
    - dnat (Deep Fake)


2. **LiDAR** - The LiDAR point cloud is the closest LiDAR scan to the camera timestamp of the core frame. Zenseact Open Dataset also provides a range of LiDAR point clouds captured in [-1s, +1s] at 10Hz around the core frame for the sequences.


3. **OXTS** - High-precision GPS. OXTS data is provided in [-1s, ~10s] around the core frames for each sequence.

#### There are 4 types of annotationed objects:  
1. **dynamic_objects** - objects that can move (vehicles, pedestrians etc.) - annotated with 2D/3D bounding boxes
2. **static_objects** - non-movable objects (light poles, traffic signs etc.) - annotated with 2D/3D bounding boxes
3. **lane_markings** - lane markings and road paitings - annotated with polygons
4. **ego_road** (Doesn't exist for all frames) - polygons that shows the road where ego vehicle can drive - annotated with polygons 

# Initialization


In [ ]:
# import the ZOD DevKit
from zod import ZodFrames
from zod import ZodSequences

# import default constants
import zod.constants as constants
from zod.constants import Camera, Lidar, Anonymization, AnnotationProject

# set path to dataset and choose version
data_dir = "/staging/dataset_donation/round_2"
version = "mini"  # "mini" or "full"

# initialize ZodFrames
zod_frames = ZodFrames(dataset_root=data_dir, version=version)

# initialize ZodSequences
zod_sequences = ZodSequences(dataset_root=data_dir, version=version)

### Split into Training and Validation sets

In [ ]:
# get default training and validation splits
training_frames = zod_frames.get_split(constants.TRAIN)
validation_frames = zod_frames.get_split(constants.VAL)

# print the number of training and validation frames
print(f"Number of training frames: {len(training_frames)}")
print(f"Number of validation frames: {len(validation_frames)}")

training_sequences = zod_sequences.get_split(constants.TRAIN)
validation_sequences = zod_sequences.get_split(constants.VAL)
print(f"Number of training sequences: {len(training_sequences)}")
print(f"Number of validation sequences: {len(validation_sequences)}")

# print out the first 5 training frames
print("The 5 first training frames have the ids:", training_frames[:5])

# show the first training sequence
print("The first training sequence has the id:", training_sequences[0])

### Fetch a ZodFrame
The ZodFrames class yeild a `ZodFrame` which acts a cache for the light-weight data (e.g., ego-motion, calibration, and metadata), but also holds an `info` attribute. This in turn holds all the paths to more heavy-weight data (e.g., images and point clouds).


In [ ]:
# we can get a specific frame by its id
frame_from_id = zod_frames["009158"]
# or via the index
frame_from_idx = zod_frames[9158]

# these two frames are the same
assert frame_from_id.info == frame_from_idx.info

### Look at some data within a ZodFrame

In [ ]:
zod_frame = zod_frames[62592]

# we can access the metadata of a frame
metadata = zod_frame.metadata

# print a subsample of meta data
print(f"Frame id: {metadata.frame_id}")
print(f"Country Code: {metadata.country_code}")
print(f"Time of day: {metadata.time_of_day}")
print(f"Number of vehicles in the frame: {metadata.num_vehicles}")

In [ ]:
# we can use the frame to get the ego-motion of our the vehicle
ego_motion = zod_frame.ego_motion
print(f"Acceleration: {ego_motion.accelerations.shape}")
print(f"Velocities: {ego_motion.velocities.shape}")
print(f"Poses: {ego_motion.poses.shape}")
print(f"Timestamps: {ego_motion.timestamps.shape}")

In [ ]:
# note that the ego-motion is a lightwieght version of the oxts data
oxts = zod_frame.oxts
print(f"Acceleration: {oxts.accelerations.shape}")
print(f"Velocities: {oxts.velocities.shape}")
print(f"Poses: {oxts.poses.shape}")
print(f"Timestamps: {oxts.timestamps.shape}")

In [ ]:
# we can also get the calibrations
calibrations = zod_frame.calibration

print(calibrations.lidars[Lidar.VELODYNE])
print(calibrations.cameras[Camera.FRONT])

#### Camera Data 

In [ ]:
# get the camera core-frame from front camera with dnat anonymization
camera_core_frame = zod_frame.info.get_key_camera_frame(Anonymization.DNAT)
print(camera_core_frame)

In [ ]:
from matplotlib import pyplot as plt

%matplotlib inline
plt.rcParams["figure.figsize"] = [20, 10]

# one can read the image from the filepath
image = camera_core_frame.read()
# or use a helper directly from the frame
zod_frame.get_image(Anonymization.DNAT)

plt.axis("off")
plt.imshow(image)

#### LiDAR Data
Lidar fields description:

| Name | Type | Units | Description |
| --- | --- | --- | --- |
| 'timestamp' | string |  seconds  | UTC timestamp of each point. |
| 'x' | double |  meters  | x coordinate of the point in lidar frame |
| 'y' | double |  meters  | y coordinate of the point in lidar frame |
| 'z' | double |  meters  | z coordinate of the point in lidar frame |
| 'intensity' | double |    | intensity level of each point in range [0..255] |
| 'diode_index' | integer |    | index of diode emitter which produced a point (1..128) |

In [ ]:
zod_frame = zod_frames[62592]

# get the lidar core-frame
lidar_core_frame = zod_frame.info.get_key_lidar_frame()
print(lidar_core_frame)

In [ ]:
# Load the lidar data
pc = lidar_core_frame.read()

# This returns a zod LidarData dataclass, which is a wrapper around several numpy arrays
from zod.zod_dataclasses import LidarData
assert isinstance(pc, LidarData)

# Alternatively, we can use helper functions on the frame itself
assert zod_frame.get_lidar_data()[0] == pc
assert zod_frame.get_lidar_frames()[0].read() == pc

print(f"Points: {pc.points.shape}")  # x, y, z
print(f"Timestamps: {pc.timestamps.shape}")
print(f"Intensity: {pc.intensity.shape}")
print(f"Diode: {pc.diode_idx.shape}")

#### Annotations

In [ ]:
# get a new frame
zod_frame = zod_frames["082291"]

# get the object annotations
annotations = zod_frame.get_annotation(AnnotationProject.OBJECT_DETECTION)

# get a single annotation object by index
idx = 31
print(f"Annotation: {annotations[idx].name}")

# there are both 2d and 3d annotations
annotation_2d = annotations[idx].box2d
annotation_3d = annotations[idx].box3d
print(annotation_2d)
print(annotation_3d)

In [ ]:
from zod.visualization.object_visualization import overlay_object_2d_box_on_image
from zod.visualization.object_visualization import overlay_object_3d_box_on_image

# we can overlay the 2d annotation on the front camera image
camera_core_frame = zod_frame.info.get_key_camera_frame(Anonymization.DNAT)
image = plt.imread(camera_core_frame.filepath)

image = overlay_object_2d_box_on_image(image, annotation_2d, color=(255, 0, 0), line_thickness=10)

plt.figure()
plt.axis("off")
plt.imshow(image)

# we can also overlay the 3d annotation on the front camera image,
# but for this we also need the calibrations of the sensor
calibrations = zod_frame.calibration

# overlay the 3d box on the image
image = overlay_object_3d_box_on_image(
    image, annotation_3d, calibrations, color=(255, 0, 0), line_thickness=10
)

plt.figure()
plt.axis("off")
plt.imshow(image)

In [ ]:
from zod.frames.polygon_annotations.polygon_transformations import polygons_to_binary_mask

zod_frame = zod_frames[9158]

# get the ego road annotations
polygon_annotations = zod_frame.get_annotation(AnnotationProject.EGO_ROAD)

# convert the polygons to a binary mask (which can be used
# for ground truth in e.g. semantic segmentation)
mask = polygons_to_binary_mask(polygon_annotations)

# visualize the mask
plt.axis("off")
plt.imshow(mask)

In [ ]:
# get another frame
zod_frame = zod_frames[23996]

# get the lane markings annotations
project = constants.AnnotationProject.LANE_MARKINGS
polygon_annotations = zod_frame.get_annotation(project)

# convert the polygons to a binary mask
mask = polygons_to_binary_mask(polygon_annotations)

# visualize the mask
plt.axis("off")
plt.imshow(mask)

In [ ]:
# We can overlay the ego road annotations on the image
from zod.visualization.polygon_utils import overlay_mask_on_image
from zod.frames.polygon_annotations.polygon_transformations import polygons_to_binary_mask

zod_frame = zod_frames[9158]

# get the camera core-frame from front camera with dnat anonymization
camera_core_frame = zod_frame.info.get_key_camera_frame(Anonymization.DNAT)

# get the image
image = plt.imread(camera_core_frame.filepath)

# get the ego road annotations
polygon_annotations = zod_frame.get_annotation(AnnotationProject.EGO_ROAD)

# convert the polygons to a binary mask (which can be used
# for ground truth in e.g. semantic segmentation)
mask = polygons_to_binary_mask(polygon_annotations)

# overlay the mask on the image
image = overlay_mask_on_image(mask, image, fill_color=(100, 0, 0), alpha=0.5)

# visualize the mask
plt.axis("off")
plt.imshow(image)

In [ ]:
# we can overlay the lane markings annotations on the image
zod_frame = zod_frames[29229]

# get the camera core-frame from front camera with dnat anonymization
camera_core_frame = zod_frame.info.get_key_camera_frame(Anonymization.DNAT)

# get the image
fp = camera_core_frame.filepath.replace(
    "/staging/dataset_donation/round_2", "/Users/s0001621/data/zod"
)
image = plt.imread(fp)

# get the ego road annotations
polygon_annotations = zod_frame.get_annotation(AnnotationProject.LANE_MARKINGS)

# convert the polygons to a binary mask (which can be used
# for ground truth in e.g. semantic segmentation)
mask = polygons_to_binary_mask(polygon_annotations)

# overlay the mask on the image
image = overlay_mask_on_image(mask, image, fill_color=(0, 0, 100), alpha=0.75)

# visualize the mask
plt.axis("off")
plt.imshow(image)

In [ ]:
# Visualize LiDAR and objects in Bird's Eye View
from zod.visualization.lidar_bev import BEVBox
from zod.zod_dataclasses import LidarData

zod_frame = zod_frames["009158"]

# get the LiDAR point cloud
pcd = zod_frame.get_lidar_data()[0]

# get the object annotations
object_annotations = zod_frame.get_annotation(AnnotationProject.OBJECT_DETECTION)

import numpy as np
bev = BEVBox()
bev_image = bev(
    np.hstack((pcd.points, pcd.intensity[:, None])),
    (
        np.array([obj.name for obj in object_annotations if obj.box3d]),
        np.concatenate(
            [obj.box3d.center[None, :] for obj in object_annotations if obj.box3d], axis=0
        ),
        np.concatenate(
            [obj.box3d.size[None, :] for obj in object_annotations if obj.box3d], axis=0
        ),
        [obj.box3d.orientation for obj in object_annotations if obj.box3d],
    ),
)

In [ ]:
# we can also visualize the lidar point cloud in the image
from zod.visualization.lidar_on_image import visualize_lidar_on_image
from zod.zod_dataclasses import LidarData

zod_frame = zod_frames["087912"]

image = zod_frame.get_image()

# Plot single Lidar point cloud
core_lidar = zod_frame.get_lidar_data()[0]
lid_image = visualize_lidar_on_image(
    core_lidar, 
    zod_frame.calibration, 
    image,
)
plt.axis("off")
plt.imshow(lid_image)
plt.show()

# Plot aggregated Lidar point cloud
aggregated_lidar = zod_frame.get_aggregated_point_cloud(num_before=10, num_after=0)
lid_image = visualize_lidar_on_image(
    aggregated_lidar, 
    zod_frame.calibration, 
    image,
)
plt.axis("off")
plt.imshow(lid_image)
plt.show()

In [ ]:
# we can also visualize all together
zod_frame = zod_frames[9158]

pcd = zod_frame.get_aggregated_point_cloud(num_before=3)
annotations = zod_frame.get_annotation(AnnotationProject.OBJECT_DETECTION)
polygon_annotations = zod_frame.get_annotation(AnnotationProject.EGO_ROAD)
mask = polygons_to_binary_mask(polygon_annotations)
calibrations = zod_frame.calibration
image = zod_frame.get_image(Anonymization.DNAT)

# overlay the mask/annotation/pointcloud on the image
image = visualize_lidar_on_image(pcd, calibrations, image)
image = overlay_mask_on_image(mask, image, fill_color=(100, 0, 0), alpha=0.5)
for annotation in annotations:
    if annotation.box3d:
        image = overlay_object_3d_box_on_image(
            image, annotation.box3d, calibrations, color=(0, 100, 0), line_thickness=10
        )
plt.axis("off")
plt.imshow(image)
plt.show()

# END OF STUFF THAT ACTUALLY WORKS #

In [ ]:
raise NotImplementedError("now stuff breaks")

### Visualize OXTS

In [ ]:
if False:
    from zod.visualization.oxts_visualization import plot_gps_track_from_dataset_sequence

    frame_id = "029229"
    ego_motion = zod.read_ego_motion(frame_id)

    # plot GPS track on interactive map
    plot_gps_track_from_dataset_sequence(ego_motion)

In [ ]:
# Todo: fix this
if False:
    from zod.visualization.oxts_on_image import visualize_gps_on_image
    import cv2

    # visualize GPS track over image
    timestamp = zod.get_timestamp(frame_id)
    camera_calib = zod.read_calibration(frame_id).cameras["camera_front"]

    gps_on_image = visualize_gps_on_image(ego_motion, timestamp, camera_calib, image)
    gps_on_image = cv2.cvtColor(gps_on_image, cv2.COLOR_BGR2RGB)
    plt.imshow(gps_on_image)
    plt.title("GPS on image")
    plt.show()

#### Visualize all objects (both static and dynamic) in a scene

In [ ]:
from zod.visualization.object_visualization import (
    overlay_object_2d_box_on_image,
    overlay_object_properties_on_image,
)

frame_id = "018591"

object_annotations = zod.read_object_detection_annotation(frame_id)

image = plt.imread(zod.get_image_path(frame_id))

text_areas = []
for object_index, object_annotation in enumerate(object_annotations):
    image = overlay_object_2d_box_on_image(image, object_annotation.box2d)
    image = overlay_object_properties_on_image(
        image, object_annotation, properties_list=["name"], color=(255, 255, 0), text_areas=[]
    )

plt.axis("off")
plt.imshow(image)

#### Visualize only vehicles

In [ ]:
from zod.visualization.object_visualization import (
    overlay_object_2d_box_on_image,
    overlay_object_properties_on_image,
)

frame_id = "018591"

object_annotations = zod.read_object_detection_annotation(frame_id)

image = plt.imread(zod.get_image_path(frame_id))

for object_index, object_annotation in enumerate(object_annotations):
    text_areas = []
    if object_annotation.name == "Vehicle":
        image = overlay_object_2d_box_on_image(image, object_annotation.box2d)
        image = overlay_object_properties_on_image(
            image,
            object_annotation,
            properties_list=["object_id"],
            color=(255, 255, 0),
            object_id=object_index,
            text_areas=[],
        )

plt.axis("off")
plt.imshow(image)

#### Visualize only pedestrians

In [ ]:
from zod.visualization.object_visualization import (
    overlay_object_2d_box_on_image,
    overlay_object_properties_on_image,
)

frame_id = "062592"

object_annotations = zod.read_object_detection_annotation(frame_id)

image = plt.imread(zod.get_image_path(frame_id))

for object_index, object_annotation in enumerate(object_annotations):
    text_areas = []
    if object_annotation.name == "Pedestrian":
        image = overlay_object_2d_box_on_image(image, object_annotation.box2d)
        image = overlay_object_properties_on_image(
            image,
            object_annotation,
            properties_list=["object_id"],
            color=(255, 255, 0),
            object_id=object_index,
            text_areas=[],
        )

plt.axis("off")
plt.imshow(image)

#### Visualize only pole objects

In [ ]:
from zod.visualization.object_visualization import (
    overlay_object_2d_box_on_image,
    overlay_object_properties_on_image,
)

frame_id = "009158"

object_annotations = zod.read_object_detection_annotation(frame_id)

image = plt.imread(zod.get_image_path(frame_id))

for object_index, object_annotation in enumerate(object_annotations):
    text_areas = []
    if object_annotation.name == "PoleObject":
        image = overlay_object_2d_box_on_image(image, object_annotation.box2d)
        image = overlay_object_properties_on_image(
            image,
            object_annotation,
            properties_list=["object_type"],
            color=(255, 0, 0),
            text_areas=[],
        )

plt.axis("off")
plt.imshow(image)

#### Visualize only Traffic Signs

In [ ]:
from zod.visualization.object_visualization import (
    overlay_object_2d_box_on_image,
    overlay_object_properties_on_image,
)

frame_id = "062592"

object_annotations = zod.read_object_detection_annotation(frame_id)

image = plt.imread(zod.get_image_path(frame_id))

for object_index, object_annotation in enumerate(object_annotations):
    text_areas = []
    if object_annotation.name == "TrafficSign":
        image = overlay_object_2d_box_on_image(image, object_annotation.box2d)
        image = overlay_object_properties_on_image(
            image,
            object_annotation,
            properties_list=["object_id"],
            color=(255, 0, 0),
            text_areas=[],
            object_id=object_index,
        )

plt.axis("off")
plt.imshow(image)

### Visualize lane marking annotations

In [ ]:
from zod.visualization.lane_markings_visualization import overlay_lane_markings_on_image

frame_id = "029229"
lane_markings_annotation = zod.read_lane_markings_annotation(frame_id)
image = plt.imread(zod.get_image_path(frame_id))
image = overlay_lane_markings_on_image(lane_markings_annotation, image)

plt.axis("off")
plt.imshow(image)

### Ego Road annotations

In [ ]:
from zod.visualization.ego_road_visualization import overlay_ego_road_on_image

frame_id = "062592"
ego_road_annotation = zod.read_ego_road_annotation(frame_id)
image = plt.imread(zod.get_image_path(frame_id))
image = overlay_ego_road_on_image(ego_road_annotation, image)

plt.axis("off")
plt.imshow(image)